In [ ]:
"""
Created on Thu Jan 16 14:55 2023

Trying to do overview plots like Pierre

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

import basal_melt_NEMO.metrics_functions as mf

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'
mask_path = '/data/cburgard/TOOLS/'

In [5]:
### FILE T PREPARATION

In [6]:
file_open_list = []
file_closed_list = []

for yy in range(9):
    yy1=yy+1
    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
file_T_open_all = xr.concat(file_open_list, dim='time_counter')
file_T_closed_all = xr.concat(file_closed_list, dim='time_counter')


In [7]:
file_T_open_all = file_T_open_all.rename({'time_counter':'time'}) 
file_T_open_all['time'] = file_T_open_all['time'].dt.year
file_T_open_all = file_T_open_all.where(file_T_open_all['so'] > 0)

file_T_closed_all = file_T_closed_all.rename({'time_counter':'time'}) 
file_T_closed_all['time'] = file_T_closed_all['time'].dt.year

In [8]:
mask_land_closed = file_T_closed_all['so'].isel(time=0,deptht=0)
mask_land_open = file_T_open_all['so'].isel(time=0)

In [9]:
cellarea_open = file_T_open_all['area'].isel(time=0).where(mask_land_open > 0).drop('time')
cellarea_closed = file_T_closed_all['area'].isel(time=0).where(mask_land_closed > 0).drop('time')

In [10]:
### FILE U PREPARATION

In [11]:
file_open_list = []
file_closed_list = []

for yy in range(9):
    yy1=yy+1

    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_U.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_U.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
    
file_U_open_all = xr.concat(file_open_list, dim='time_counter')
file_U_closed_all = xr.concat(file_closed_list, dim='time_counter')

file_U_open_all = file_U_open_all.rename({'time_counter':'time'}) 
file_U_open_all['time'] = file_U_open_all['time'].dt.year

file_U_closed_all = file_U_closed_all.rename({'time_counter':'time'}) 
file_U_closed_all['time'] = file_U_closed_all['time'].dt.year

In [12]:
u_vertsum_open = (file_U_open_all['uocetr_eff'] * (mask_land_open.rename({'deptht':'depthu'}) > 0)).sum('depthu')
u_vertsum_closed = (file_U_closed_all['uocetr_eff'] * (mask_land_closed.rename({'deptht':'depthu'}) > 0)).sum('depthu')

In [13]:
### FILE V PREPARATION

In [14]:
file_open_list = []
file_closed_list = []

for yy in range(9):
    yy1=yy+1

    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_V.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_V.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
    
file_V_open_all = xr.concat(file_open_list, dim='time_counter')
file_V_closed_all = xr.concat(file_closed_list, dim='time_counter')

file_V_open_all = file_V_open_all.rename({'time_counter':'time'}) 
file_V_open_all['time'] = file_V_open_all['time'].dt.year

file_V_closed_all = file_V_closed_all.rename({'time_counter':'time'}) 
file_V_closed_all['time'] = file_V_closed_all['time'].dt.year

In [15]:
### SEA ICE FILE PREPARATION

In [16]:
file_open_list = []
file_closed_list = []

for yy in range(9):

    file_open = xr.open_mfdataset(inputpath_open+'/n42openc_00'+str(yy)+'10101_00'+str(yy+1)+'01231_1M_icemod.nc')
    file_closed = xr.open_mfdataset(inputpath_closed+'/n42tm21_00'+str(yy)+'10101_00'+str(yy+1)+'01231_1M_icemod.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)

file_ice_open_all = xr.concat(file_open_list, dim='time_counter')
file_ice_closed_all = xr.concat(file_closed_list, dim='time_counter')


In [17]:
### MASK FILE

In [18]:
ocean_masks = xr.open_dataset(mask_path + 'basin_masks_orca1_nemo4p2.nc')

In [19]:
### VAR LIST

In [20]:
var_ocean_list_in = ['global_SST', 'wed_gyre','ross_gyre','ACC',
            #'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
            'Sbot_WWED','Sbot_EWED','Sbot_WROSS','Sbot_EROSS','Sbot_AMU',
            'Tbot_WWED','Tbot_EWED','Tbot_WROSS','Tbot_EROSS','Tbot_AMU',
            'OHC_tot','OHC_700','OHC_2000',
            'AMOC_26N','AMOC_30S']

var_seaice_list_in = ['mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
                      'mar_sia_arc','sep_sia_arc','feb_sia_ant','sep_sia_ant',
                     'mar_siv_arc','sep_siv_arc','feb_siv_ant','sep_siv_ant']

In [21]:
var_ds_all = mf.compute_VALSO_ocean_metrics(var_ocean_list_in, 
                                       ['closed_isf','open_isf'], 
                                      [file_T_closed_all, file_T_open_all], 
                                      [file_U_closed_all, file_U_open_all], 
                                      [file_V_closed_all, file_V_open_all], 
                                      [cellarea_closed, cellarea_open], 
                                         ocean_masks, 
                                         smallest_domain=True)

f = mf.VALSO_plot_ocean_comparison(var_ocean_list_in, 
                        var_ds_all,
                      ['closed_isf','open_isf'], 
                      ['deepskyblue','royalblue'])
f.show()

USING CELL AREA FROM RUN NUMBER 1 IN THE LIST
closed_isf
Computing global SST
Computing ACC
Computing gyres
Computing bottom properties
Computing ocean heat content
open_isf
Computing global SST
Computing ACC
Computing gyres
Computing bottom properties
Computing ocean heat content


libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [ ]:
var_ds_seaice = mf.compute_VALSO_seaice_metrics(var_seaice_list_in, 
                                        ['closed_isf','open_isf'], 
                                        [file_T_closed_all, file_T_open_all], 
                                        [file_ice_closed_all, file_ice_open_all], 
                                        [cellarea_closed, cellarea_open], 
                                        ['deepskyblue','royalblue'],
                                        smallest_domain=True)

f_seaice = mf.VALSO_plot_seaice_comparison(var_seaice_list_in, 
                                        var_ds_seaice, 
                                        ['closed_isf','open_isf'],  
                                        ['deepskyblue','royalblue'])

USING CELL AREA FROM RUN NUMBER 1 IN THE LIST
closed_isf
Computing sea-ice extent
Computing sea-ice area


In [ ]:
print('OK done')

In [ ]:
def var_obs_mean_std(var_list_in):

    var_list = ['global_SST','wed_gyre','ross_gyre','ACC',
                'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
                'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
                'mar_sia_arc','sep_sia_arc','feb_sia_ant','sep_sia_ant',
                'mar_siv_arc','sep_siv_arc','feb_siv_ant','sep_siv_ant',
                'Sbot_WWED','Sbot_EWED','Sbot_WROSS','Sbot_EROSS','Sbot_AMU',
                'Tbot_WWED','Tbot_EWED','Tbot_WROSS','Tbot_EROSS','Tbot_AMU',
               'AMOC_26N','AMOC_30S','OHC_tot','OHC_700','OHC_2000']
    
    var_obs_mean = xr.DataArray(data=np.array([
                20., 56.0, 20.0, 136.7, 
               155.4, 47.7, 20.7, 1500,
               15.3, 6.3, 3.3, 19.8, 
               14.4, 6.0, 2.25, 15.0,
               27., 9., 2.5, 20.,
               34.9, np.nan, 35.0, np.nan, np.nan, 
               np.nan, -1.95, -1.9, -1.7, np.nan,
               17.0, np.nan, np.nan, np.nan, np.nan]), dims='var').assign_coords({'var': var_list})

    var_obs_std = xr.DataArray(data=np.array([
                  np.nan, 8.0, 5.0, 7.8, 
                  45.0, 34.0, 67.0, 237,
                  1.0, 1.0, 0.5, 0.5, 
                  1.29, 1.49, 0.25, 0.4,
                  3, 3, 2, 5,
                  0.0, np.nan, 0.0, np.nan, np.nan,
                  np.nan, 0.2, 0.4, 0.4, np.nan,
                  10.0, np.nan, np.nan, np.nan, np.nan]), dims='var').assign_coords({'var': var_list})

    return var_obs_mean.sel(var=var_list_in), var_obs_std.sel(var=var_list_in)

In [ ]:
def bottom_prop_reg(var,reg,bottom_cell):

    # mask regions for bottom properties
    mask_regions = xr.Dataset()
    mask_regions['AMU'] = (lon >= -109.64) & (lon <= -102.23) & (lat >= -75.80) & (lat <= -71.66)
    mask_regions['WROSS'] = (lon >= 157.100) & (lon <= 173.333) & (lat >= -78.130) & (lat <= -74.040)
    mask_regions['EROSS'] = (lon >= -176.790) & (lon <= -157.820) & (lat >= -78.870) & (lat <= -77.520)
    mask_regions['WWED'] = (lon >= -65.130) & (lon <= -53.020) & (lat >= -75.950) & (lat <= -72.340)
    mask_regions['EWED'] = (lon >= -45.647) & (lon <= -32.253) & (lat >= -78.632) & (lat <= -76.899)
    
    return var.isel(deptht=bottom_cell).where(mask_regions[reg], drop=True)

In [ ]:
def compute_VALSO_ocean_metrics(var_list_in, run_name_list, vector_T_list, vector_U_list, vector_V_list, cellarea_list, ocean_masks, smallest_domain=True):

    var_ds_list = []

    if smallest_domain:
        cc_prev = 10**23
        for n,cc in enumerate(cellarea_list):
            if cc.sum() < cc_prev:
                cc_prev = cc.sum().load()
                n_small = n
                
        cell_area = cellarea_list[n_small]
        print('USING CELL AREA FROM RUN NUMBER '+str(n)+' IN THE LIST')
    else:
        print('USING CELL AREA FROM EACH RUN SEPARATELY')
        
    for n,rrun in enumerate(run_name_list):
        print(run_name_list[n])
        
        var_to_plot = xr.Dataset()

        file_T_n = vector_T_list[n]
        file_U_n = vector_U_list[n]
        file_V_n = vector_V_list[n]

        #### format cell_area
        if not smallest_domain:
            cell_area = cellarea_list[n]

        if 'deptht' in cell_area.dims:
            cell_area_here = cell_area.isel(deptht=0)
        else:
            cell_area_here = cell_area

        mask_land = file_T_n['so'].isel(time=0,deptht=0).squeeze().drop('time')

        lon = file_T_n.nav_lon
        lat = file_T_n.nav_lat


        ### prepare integrated horizontal mass transport
        if ('ACC' in var_list_in) or ('wed_gyre' in var_list_in) or ('ross_gyre' in var_list_in):
            
            u_vertsum = (file_U_n['uocetr_eff'] * (mask_land.rename({'deptht':'depthu'}) > 0)).sum('depthu').load()

        #######
        if 'global_SST' in var_list_in:
            print('Computing global SST')
            if 'deptht' in cell_area.dims:
                cell_area_here = cell_area.isel(deptht=0)
            else:
                cell_area_here = cell_area
                
            var_to_plot['global_SST'] = uf.weighted_mean(file_T_n['thetao'].isel(deptht=0),['x','y'],cell_area_here)
        
        #######
        if 'ACC' in var_list_in:
            print('Computing ACC')
            var_to_plot['ACC'] = u_vertsum.sel(x=220,y=range(79,107)).sum('y')
            var_to_plot['ACC'] = var_to_plot['ACC']/10**6


        ########
        print('Computing gyres')
        if 'wed_gyre' in var_list_in:
            uocetr_Wed = u_vertsum.where(uf.in_range(cell_area_here.nav_lat,[-66.50,-60.40]) & uf.in_range(cell_area_here.nav_lon,[-31.25,37.50]), drop=True)

            var_to_plot['wed_gyre'] = uocetr_Wed.cumsum('y').max(['y','x'])/10**6
        
        ########
        if 'ross_gyre' in var_list_in:
            uocetr_Ross = u_vertsum.where(uf.in_range(cell_area_here.nav_lat,[-72.650,-61.600]) & ((cell_area_here.nav_lon <= -135.75) | (cell_area_here.nav_lon >= 360-168.500)), drop=True)
            
            var_to_plot['ross_gyre'] = uocetr_Ross.cumsum('y').max(['y','x'])/10**6

        ########
        print('Computing bottom properties')
        mask_cells = (file_T_n['so'] > 0).isel(time=0).drop('time').drop('time_centered').load()
        bottom_cell = mask_cells.sum('deptht') - 1

        if 'Tbot_AMU' in var_list_in:
            reg = 'AMU'
            var_to_plot['Tbot_'+reg] = bottom_prop_reg(file_T_n['thetao'],reg,bottom_cell).mean(['x','y'])
            var_to_plot['Sbot_'+reg] = bottom_prop_reg(file_T_n['so'],reg,bottom_cell).mean(['x','y'])

        if 'Tbot_WROSS' in var_list_in:
            reg = 'WROSS'
            var_to_plot['Tbot_'+reg] = bottom_prop_reg(file_T_n['thetao'],reg,bottom_cell).mean(['x','y'])
            var_to_plot['Sbot_'+reg] = bottom_prop_reg(file_T_n['so'],reg,bottom_cell).mean(['x','y'])

        if 'Tbot_EROSS' in var_list_in:
            reg = 'EROSS'
            var_to_plot['Tbot_'+reg] = bottom_prop_reg(file_T_n['thetao'],reg,bottom_cell).mean(['x','y'])
            var_to_plot['Sbot_'+reg] = bottom_prop_reg(file_T_n['so'],reg,bottom_cell).mean(['x','y'])

        if 'Tbot_WWED' in var_list_in:
            reg = 'WWED'
            var_to_plot['Tbot_'+reg] = bottom_prop_reg(file_T_n['thetao'],reg,bottom_cell).mean(['x','y'])
            var_to_plot['Sbot_'+reg] = bottom_prop_reg(file_T_n['so'],reg,bottom_cell).mean(['x','y'])

        if 'Tbot_EWED' in var_list_in:
            reg = 'EWED'
            var_to_plot['Tbot_'+reg] = bottom_prop_reg(file_T_n['thetao'],reg,bottom_cell).mean(['x','y'])
            var_to_plot['Sbot_'+reg] = bottom_prop_reg(file_T_n['so'],reg,bottom_cell).mean(['x','y'])

        ########
        print('Computing ocean heat content')
        
        rho0 = 1020.
        c_p = 4000.
        
        if 'OHC_tot' in var_list_in:
            
            var_to_plot['OHC_tot'] = rho0 * c_p * (file_T_n['thetao'] * cell_area_here * file_T_n['e3t']).sum(['x','y','deptht']) /10**22

        if 'OHC_700' in var_list_in:
            
            deptht_700 = (file_T_n.deptht <= 700).sum()

            var_to_plot['OHC_700'] = rho0 * c_p * (file_T_n['thetao'] * cell_area_here* file_T_n['e3t']).isel(deptht = np.arange(deptht_700)).sum(['x','y','deptht']) /10**22

        if 'OHC_2000' in var_list_in:
            
            deptht_2000 = (file_T_n.deptht <= 2000).sum()

            var_to_plot['OHC_2000'] = rho0 * c_p * (file_T_n['thetao'] * cell_area_here * file_T_n['e3t']).isel(deptht = np.arange(deptht_2000)).sum(['x','y','deptht']) /10**22

        if 'AMOC_26N' in var_list_in:
            
            Atl_26N_mask = (lat<=26.5) & (lat>=25.5) & np.isfinite(ocean_masks['atlantic'])
            v_atlsum_26N = file_V_n['vocetr_eff'].where(Atl_26N_mask,drop=True).sum('x')
            var_to_plot['AMOC_26N'] = (v_atlsum_26N.assign_coords({'depthv': -1*file_V_n['vocetr_eff'].depthv}).cumsum('depthv') / 10**6).max(['depthv','y'])


        if 'AMOC_30S' in var_list_in:

            Atl_30S_mask = (lat<=-29.5) & (lat>=-30.5) & np.isfinite(ocean_masks['atlantic'])
            v_atlsum_30S = file_V_n['vocetr_eff'].where(Atl_30S_mask,drop=True).sum('x')
            var_to_plot['AMOC_30S'] = (v_atlsum_30S.assign_coords({'depthv': -1*file_V_n['vocetr_eff'].depthv}).cumsum('depthv') / 10**6).min(['depthv','y'])

        
        var_ds_list.append(var_to_plot.assign_coords({'run': rrun}))

    var_ds_all = xr.concat(var_ds_list, dim='run')
    return var_ds_all

In [ ]:
def VALSO_plot_ocean_comparison(var_list_in, var_ds_all, run_name_list, color_list):
    
    var_obs_mean, var_obs_std = var_obs_mean_std(var_list_in)

    var_to_plot = var_ds_all.load()
    
    f = plt.figure()
    f.set_size_inches(8.25*2, 8.25*2.2)
    
    ax={}
    
    leg_hdl = []
    
    i = 0
    
    for vv in var_list_in:
        
        ax[i] = f.add_subplot(5,6,i+1)
    
        if vv in list(var_to_plot.keys()):
            for n,nname in enumerate(run_name_list):
                ax[i].plot(var_to_plot[vv].sel(run=nname), color=color_list[n])
        
        ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
        ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)
    
        if vv[0:3] == 'OHC':
            ax[i].set_title(vv+' x 10$^{22}$ J')
        else:
            ax[i].set_title(vv)
    
        i = i+1
    #f.legend()
    #f.subplots_adjust(bottom=0.05, wspace=0.1)
    
    f.tight_layout()
    sns.despine()

    return f


In [ ]:
var_ds_all = compute_VALSO_ocean_metrics(var_ocean_list_in, 
                                       ['closed_isf','open_isf'], 
                                      [file_T_closed_all, file_T_open_all], 
                                      [file_U_closed_all, file_U_open_all], 
                                      [file_V_closed_all, file_V_open_all], 
                                      [cellarea_closed, cellarea_open], 
                                         ocean_masks, 
                                         smallest_domain=True)

f = VALSO_plot_ocean_comparison(var_ocean_list_in, 
                        var_ds_all,
                      ['closed_isf','open_isf'], 
                      ['deepskyblue','royalblue'])
f.show()

In [ ]:
def compute_VALSO_seaice_metrics(var_seaice_list_in, run_name_list, vector_T_list, vector_seaice_list, cellarea_list, ocean_masks, smallest_domain=True):

    var_ds_list = []

    if smallest_domain:
        cc_prev = 10**23
        for n,cc in enumerate(cellarea_list):
            if cc.sum() < cc_prev:
                cc_prev = cc.sum().load()
                n_small = n
                
        cell_area = cellarea_list[n_small]
        print('USING CELL AREA FROM RUN NUMBER '+str(n)+' IN THE LIST')
    else:
        print('USING CELL AREA FROM EACH RUN SEPARATELY')
        
    for n,rrun in enumerate(run_name_list):
        print(run_name_list[n])
        
        var_to_plot = xr.Dataset()

        file_T_n = vector_T_list[n]
        file_seaice_n = vector_seaice_list[n]


        #### format cell_area
        if not smallest_domain:
            cell_area = cellarea_list[n]
    
        if 'deptht' in cell_area.dims:
            cell_area_here = cell_area.isel(deptht=0).drop('deptht').load()
        else:
            cell_area_here = cell_area.load()

        mask_land = file_T_n['so'].isel(time=0,deptht=0).squeeze().drop('time')

        ########
        
        lon = cell_area_here.nav_lon
        lat = cell_area_here.nav_lat
        
        mask_Arc = (lat >= 50) 
        mask_Ant = (lat <= -50) 
        
        ########

        if ('mar_sie_arc' in var_seaice_list_in) or ('sep_sie_arc' in var_seaice_list_in) or ('feb_sie_ant' in var_seaice_list_in) or ('sep_sie_ant' in var_seaice_list_in):
            print('Computing sea-ice extent')
            
            file_ice_15 = file_seaice_n['siconc'] > 0.15

            sie_Ant = (file_ice_15.where(mask_Ant) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()
            sie_Arc = (file_ice_15.where(mask_Arc) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()

        if 'mar_sie_arc' in var_seaice_list_in:
            vvar = sie_Arc.where(sie_Arc['time.month'] == 3, drop=True).squeeze()/10**12
            var_to_plot['mar_sie_arc'] = vvar.assign_coords({'time': vvar['time.year']})
            
        if 'sep_sie_arc' in var_seaice_list_in:
            vvar = sie_Arc.where(sie_Arc['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_sie_arc'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'feb_sie_ant' in var_seaice_list_in:
            vvar = sie_Ant.where(sie_Ant['time.month'] == 2, drop=True).squeeze()/10**12
            var_to_plot['feb_sie_ant'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'sep_sie_ant' in var_seaice_list_in:
            vvar = sie_Ant.where(sie_Ant['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_sie_ant'] = vvar.assign_coords({'time': vvar['time.year']})

        ########

        sia_Ant = (file_seaice_n['siconc'].where(mask_Ant) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()
        sia_Arc = (file_seaice_n['siconc'].where(mask_Arc) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()

        print('Computing sea-ice area')
        
        if 'mar_sia_arc' in var_seaice_list_in:
            vvar = sia_Arc.where(sia_Arc['time.month'] == 3, drop=True).squeeze()/10**12
            var_to_plot['mar_sia_arc'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'sep_sia_arc' in var_seaice_list_in:
            vvar = sia_Arc.where(sia_Arc['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_sia_arc'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'feb_sia_ant' in var_seaice_list_in:
            vvar = sia_Ant.where(sia_Ant['time.month'] == 2, drop=True).squeeze()/10**12
            var_to_plot['feb_sia_ant'] = vvar.assign_coords({'time': vvar['time.year']})


        if 'sep_sia_ant' in var_seaice_list_in:
            vvar = sia_Ant.where(sia_Ant['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_sia_ant'] = vvar.assign_coords({'time': vvar['time.year']})

        ########

        siv_Ant = (file_seaice_n['sivolu'].where(mask_Ant) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()
        siv_Arc = (file_seaice_n['sivolu'].where(mask_Arc) * cell_area_here).sum(['x','y']).rename({'time_counter':'time'}).load()

        print('Computing sea-ice volume')
        
        if 'mar_siv_arc' in var_seaice_list_in:
            vvar = siv_Arc.where(siv_Arc['time.month'] == 3, drop=True).squeeze()/10**12
            var_to_plot['mar_siv_arc'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'sep_siv_arc' in var_seaice_list_in:
            vvar = siv_Arc.where(siv_Arc['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_siv_arc'] = vvar.assign_coords({'time': vvar['time.year']})

        if 'feb_siv_ant' in var_seaice_list_in:
            vvar = siv_Ant.where(siv_Ant['time.month'] == 2, drop=True).squeeze()/10**12
            var_to_plot['feb_siv_ant'] = vvar.assign_coords({'time': vvar['time.year']})
            
        if 'sep_siv_ant' in var_seaice_list_in:
            vvar = siv_Ant.where(siv_Ant['time.month'] == 9, drop=True).squeeze()/10**12
            var_to_plot['sep_siv_ant'] = vvar.assign_coords({'time': vvar['time.year']})
            
        ########
        
        var_ds_list.append(var_to_plot.assign_coords({'run': rrun}))

    var_ds_all = xr.concat(var_ds_list, dim='run')
    return var_ds_all

In [ ]:
def VALSO_plot_seaice_comparison(var_list_in, var_ds_all, run_name_list, color_list):
    
    var_obs_mean, var_obs_std = var_obs_mean_std(var_list_in)

    #return var_ds_all
    var_to_plot = var_ds_all.load()
    
    f = plt.figure()
    f.set_size_inches(8.25*2, 8.25*2.2)
    
    ax={}
    
    leg_hdl = []
    
    i = 0
    
    for vv in var_list_in:
        
        ax[i] = f.add_subplot(5,6,i+1)
    
        if vv in list(var_to_plot.keys()):
            for n,nname in enumerate(run_name_list):
                ax[i].plot(var_to_plot[vv].sel(run=nname), color=color_list[n])
        
        ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
        ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)
    
        if vv[0:3] == 'OHC':
            ax[i].set_title(vv+' x 10$^22$ J')
        else:
            ax[i].set_title(vv)
    
        i = i+1
    #f.legend()
    #f.subplots_adjust(bottom=0.05, wspace=0.1)
    
    f.tight_layout()
    sns.despine()

    return f, var_ds_all


In [ ]:
var_ds_seaice = compute_VALSO_seaice_metrics(var_seaice_list_in, 
                                        ['closed_isf','open_isf'], 
                                        [file_T_closed_all, file_T_open_all], 
                                        [file_ice_closed_all, file_ice_open_all], 
                                        [cellarea_closed, cellarea_open], 
                                        ['deepskyblue','royalblue'],
                                        smallest_domain=True)

f_seaice = VALSO_plot_seaice_comparison(var_seaice_list_in, 
                                        var_ds_seaice, 
                                        ['closed_isf','open_isf'],  
                                        ['deepskyblue','royalblue'])

In [ ]:
var_to_plot = xr.Dataset()

In [ ]:
### ICE SHELF MELT
T_grid_vars_open = xr.open_dataset('/data/cburgard/CASIMIR_SIMU/interim/CDO_PROCESSED/n42openc/n42openc_0-100_1Y_grid_isfvars.nc')

lon = T_grid_vars_open.nav_lon
lat = T_grid_vars_open.nav_lat

T_grid_vars_open = T_grid_vars_open.rename({'time_counter':'time'}) 
T_grid_vars_open['time'] = T_grid_vars_open['time'].dt.year

mask_FRIS = (lon <= -30) & (lon >= -80.) & (lat >= -84.) & (lat <= -74.)
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)
mask_LarsenC = (lon <= -58) & (lon >= -66) & (lat >= -71.) & (lat <= -64.)

In [ ]:
melt_all = (T_grid_vars_open['iceshelf_cav'] * cellarea_open.max('deptht') * yearinsec *10**(-12))
FRIS_melt= melt_all.where(mask_FRIS).sum(['x','y'])
Ross_melt = melt_all.where(mask_Ross).sum(['x','y'])
LarsenC_melt = melt_all.where(mask_LarsenC).sum(['x','y'])

var_to_plot['FRIS_melt'] = xr.concat([FRIS_melt.assign_coords({'cav':'open'}), FRIS_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['Ross_melt'] = xr.concat([Ross_melt.assign_coords({'cav':'open'}), Ross_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['LarsenC_melt'] = xr.concat([LarsenC_melt.assign_coords({'cav':'open'}), LarsenC_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')

var_to_plot['total_melt'] = var_to_plot['FRIS_melt'] + var_to_plot['Ross_melt'] + var_to_plot['LarsenC_melt'] + (T_grid_vars_open['iceshelf'] * cellarea_open.max('deptht') * yearinsec * 10**(-12)).sum(['x','y'])


In [ ]:
### AMOC (density binned)


In [ ]:
def compute_sigma2(da):
    S = da['so']
    T = da['thetao']
    return gsw.density.sigma2(S, T)

In [ ]:
sigma2_closed = compute_sigma2(file_T_closed_all)
sigma2_open = compute_sigma2(file_T_open_all)

sigma2_closed_v = ((sigma2_closed + sigma2_closed.shift(deptht=1)) * 0.5).bfill('deptht').rename({'deptht':'depthv'})
sigma2_open_v = ((sigma2_open + sigma2_open.shift(deptht=1)) * 0.5).bfill('deptht').rename({'deptht':'depthv'})

In [ ]:
vocetr_eff_closed = file_V_closed_all['vocetr_eff']
vocetr_eff_open = file_V_open_all['vocetr_eff']

In [ ]:
cellvol_closed = file_V_closed_all['e3v'] * cellarea_closed
cellvol_open = file_V_open_all['e3v'] * cellarea_open

In [ ]:
celldep_closed = 

In [ ]:
cellvol_closed.isel(time=0).sum().load()

In [ ]:
var_to_plot = var_to_plot.load()

In [ ]:
f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2.2)

ax={}

leg_hdl = []

i = 0


for vv in var_list:
    
    ax[i] = f.add_subplot(5,6,i+1)

    if vv in list(var_to_plot.keys()):
        ax[i].plot(var_to_plot[vv].sel(cav='open'), color='deepskyblue')
        ax[i].plot(var_to_plot[vv].sel(cav='closed'), color='royalblue')
    
    ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
    ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)

    if vv[0:3] == 'OHC':
        ax[i].set_title(vv+' x 10$^22$ J')
    else:
        ax[i].set_title(vv)

    i = i+1
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'VALSO_comparison_plot.pdf')


In [ ]:
T_grid_vars_closed['thetao'].isel(deptht=bottom_closed).isel(time_counter=0).plot()

In [ ]:
uocetr_Ross_open = u_open['uocetr_eff'].where(uf.in_range(u_open.nav_lat,[-72.650,-61.600]) & ((u_open.nav_lon <= -135.75) | (u_open.nav_lon >= 360-168.500)), drop=True)


In [ ]:
uocetr_Ross_open.nav_lat.max()

CHECK THAT I AM LOOKING AT THE RIGHT REGIONS FOR THE GYRES

In [ ]:
ref = uocetr_Ross_open.isel(time_counter=0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_closed.where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -20], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_open.max('deptht')
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_closed.where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.NorthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_open.isel(deptht=0)
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.NorthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = mask_regions['WROSS']
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
u_open['uocetr_eff'].sel(x=range(220,221),y=range(79,107)).isel(time_counter=0)

In [ ]:

ref = T_grid_vars_closed_mean10['thetao'].isel(deptht=0).where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.PlateCarree(central_longitude=0.0, globe=None)
#wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

#theta = np.linspace(0, 2*np.pi, 100)
#center, radius = [0.5, 0.5], 0.5
#verts = np.vstack([np.sin(theta), np.cos(theta)]).T
#circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(lon,lat,ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
#ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
#ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
T_grid_vars_closed_mean10['so'].isel(deptht=0).plot()

In [ ]:
mask_land_closed = T_grid_vars_closed_mean10['so'].isel(deptht=0)

In [ ]:
mask_land_closed.plot()

In [ ]:
cellarea_closed.where(mask_land_closed)